### Homework 2020/11/01, Roger Wang

# Mission 1
請至環保署空品網下載個人所在地區2019 年逐時資料
挑選其最近的觀測站讀入 列出 監測記錄總天數 與 測項總項數

In [1]:
import pandas as pd
import numpy as np

def data_clean(tdf):
    new_col = dict(zip(tdf.columns, tdf.columns.str.strip(' '))) 
    tdf.rename(columns=new_col, inplace=True)

    for col in df.columns:
        tdf[col] = tdf[col].str.replace('NA','0')
        if col == '測項':
            tdf[col] = tdf[col].str.strip(' \n')
        if (col != '測站') and (col != '日期') and (col != '測項'):
            tdf[col] = tdf[col].str.strip(' *#ANx')
            tdf[col] = tdf[col].astype('float', copy=False, errors='ignore')
    return tdf
    
df = pd.read_csv('湖口_2019.csv', encoding='big5')

df = data_clean(df)

gp = df[['測項']].groupby(by=df['日期'])
display(gp.count())
print('監測記錄總天數:{}天'.format(len(list(gp))))
print('測項總項數:{}項'.format(gp.count().iloc[0,0]))

,測項
日期,
2019/01/01 00:00:00,15
2019/01/02 00:00:00,15
2019/01/03 00:00:00,15
2019/01/04 00:00:00,15
2019/01/05 00:00:00,15
...,...
2019/12/27 00:00:00,15
2019/12/28 00:00:00,15
2019/12/29 00:00:00,15


監測記錄總天數:365天
測項總項數:15項


## Mission 2
<img style="float: left;" src="mission2.png" width="30%"><br>
<br>
請將讀入資料集欄位
(Column)轉換為下列型態：
西元年(YYYY)、月(MM)、日(DD)、時(HH)、觀測站及各測項類別

溫度:AMB_TEMP(℃),一氧化碳:CO(ppm),一氧化氮:NO(ppb),二氧化氮:NO2(ppb),氮氧化物:NOx(ppb),臭氧:O3(ppb)
懸浮微粒:PM10(μg/m3),細懸浮微粒:PM2.5(μg/m3),降雨量:RAINFALL(mm/h),相對濕度:RH(percent),二氧化硫:SO2(ppb)
小時風向值:WD_HR(degrees),風向：WIND_DIREC,風速:WIND_SPEED(m/sec),小時風速值:WS_HR(m/sec)

In [ ]:
df1 = df.copy()
hr_cols=[ '00','01','02','03','04','05','06','07','08','09',
          '10','11','12','13','14','15','16','17','18','19',
          '20','21','22','23']
gp1 = df1.groupby('測項')
df_transform = pd.DataFrame()
for i in gp1:
    station = i[0]
    df_temp = i[1]
    for j in range(len(df_temp)):
        for hr in range(0,24):
            idx = j*24+hr
            if(df_transform.columns.shape[0]<df_temp.shape[0]):
                df_transform.loc[idx,'西元年'] = df_temp.iloc[j,1][:10].split('/')[0]
                df_transform.loc[idx,'月'] = df_temp.iloc[j,1][:10].split('/')[1]
                df_transform.loc[idx,'日'] = df_temp.iloc[j,1][:10].split('/')[2]
                df_transform.loc[idx,'時'] = hr_cols[hr]
                df_transform.loc[idx,'觀測站'] = df_temp.iloc[j,0]
            df_transform.loc[idx, station] = df_temp.iloc[j, 3+hr]
    
display(df_transform)

## Mission 3
<img style="float: left;" src="mission3.png" width="20%"><br>
<br>
請參照空品網即時 空氣品質指標(AQI) 計算公式求取逐時 AQI 指數值
並新設欄位Alert(0/1) 
<br>
註明 紫爆(AQI>200) 以上 的記錄 (Alert==1)

In [ ]:
def cal_aqi(value, item):
    if(item == 'O3' and value > 300):
        item = 'O3_8h'
    for i in range(1,8):
        low = aqi.loc[item,'low{}'.format(i)]
        high = aqi.loc[item,'high{}'.format(i)]
        if (value>=low and value<=high):
            break
    aqi_h = aqi.loc['AQI','high{}'.format(i)]
    aqi_l = aqi.loc['AQI','low{}'.format(i)]
    ret_aqi=(((aqi_h-aqi_l) / (high-low)) *(value-low))+aqi_l
    return ret_aqi
    
aqi = pd.read_csv('aqi.csv')
aqi.set_index('item', inplace=True)
df2 = df_transform.copy()
df2.drop(['AMB_TEMP','NO','NOx','RAINFALL','RH','WD_HR','WIND_DIREC','WIND_SPEED','WS_HR'], axis=1, inplace=True)
df2['O3_8h'] = df2['O3'].rolling(8).mean()
for i in range(df2.shape[0]):
    aqi_list=[]
    for j in range(5,12):
        item = df2.columns[j]
        aqi_j = cal_aqi(df2.loc[i,item], item)
        aqi_list.append(aqi_j)
    df2.loc[i,'AQI'] = round(max(aqi_list),2)

df2['Alert']=df2['AQI'].apply(lambda x: 1 if x > 200 else 0)

display(df2)
df3 = df2[df2['Alert']==1]
print('有{}天發生紫爆'.format(df3.shape[0]))
display(df3)

In [ ]:
df2['O3_8h'] = round(df2['O3_8h'],2)
df2.to_csv('hukou_aqi_2019.csv')